In [1]:
import duckdb

In [2]:
con = duckdb.connect("database.ddb")
print("Connected to DuckDB")

Connected to DuckDB


In [3]:
con.execute("""
CREATE TABLE characters(
  name VARCHAR,
  affiliation VARCHAR,  -- 'Jedi','Sith','Rebel','Empire', etc.
  species VARCHAR,
  homeworld VARCHAR,
  midichlorians INTEGER
);
""")

con.execute("""
INSERT INTO characters VALUES
  ('Luke Skywalker','Jedi','Human','Tatooine',15000),
  ('Leia Organa','Rebel','Human','Alderaan',4000),
  ('Han Solo','Rebel','Human','Corellia',3000),
  ('Darth Vader','Sith','Human','Tatooine',20000),
  ('Yoda','Jedi','Unknown','Dagobah',17500),
  ('Obi-Wan Kenobi','Jedi','Human','Stewjon',16000),
  ('Emperor Palpatine','Sith','Human','Naboo',19000);
""")

con.execute("""
CREATE TABLE planets(
  name VARCHAR,
  climate VARCHAR,
  region VARCHAR
);
""")

con.execute("""
INSERT INTO planets VALUES
  ('Tatooine','desert','Outer Rim'),
  ('Alderaan','temperate','Core'),
  ('Corellia','temperate','Core'),
  ('Dagobah','swamp','Outer Rim'),
  ('Stewjon','temperate','Unknown'),
  ('Naboo','temperate','Mid Rim'),
  ('Coruscant','urban','Core');
""")

print("Seeded tables: characters & planets")

Seeded tables: characters & planets


### Basic Select & Filtering

In [4]:
print(con.execute("SELECT * FROM characters LIMIT 5").fetchall())

# Only Jedi
print(
    con.execute("""
  SELECT name, homeworld, midichlorians
  FROM characters
  WHERE affiliation = 'Jedi'
  ORDER BY midichlorians DESC
""").fetchall()
)

# Case-insensitive search (ILIKE)
print(
    con.execute("""
  SELECT name, affiliation
  FROM characters
  WHERE name ILIKE '%sky%'
""").fetchall()
)

[('Luke Skywalker', 'Jedi', 'Human', 'Tatooine', 15000), ('Leia Organa', 'Rebel', 'Human', 'Alderaan', 4000), ('Han Solo', 'Rebel', 'Human', 'Corellia', 3000), ('Darth Vader', 'Sith', 'Human', 'Tatooine', 20000), ('Yoda', 'Jedi', 'Unknown', 'Dagobah', 17500)]
[('Yoda', 'Dagobah', 17500), ('Obi-Wan Kenobi', 'Stewjon', 16000), ('Luke Skywalker', 'Tatooine', 15000)]
[('Luke Skywalker', 'Jedi')]


### Aggregation

In [5]:
# Count by affiliation
print(
    con.execute("""
  SELECT affiliation, COUNT(*) AS n
  FROM characters
  GROUP BY affiliation
  ORDER BY n DESC
""").fetchall()
)

# Average midichlorians by affiliation (demo)
print(
    con.execute("""
  SELECT affiliation, AVG(midichlorians)::INTEGER AS avg_m_count
  FROM characters
  GROUP BY affiliation
  ORDER BY avg_m_count DESC
""").fetchall()
)

[('Jedi', 3), ('Rebel', 2), ('Sith', 2)]
[('Sith', 19500), ('Jedi', 16167), ('Rebel', 3500)]


### Join with planets

In [6]:
# Characters with planet climate & region
print(
    con.execute("""
  SELECT c.name, c.affiliation, c.homeworld, p.climate, p.region
  FROM characters c
  LEFT JOIN planets p ON c.homeworld = p.name
  ORDER BY c.name
""").fetchall()
)

# How many characters by planet climate?
print(
    con.execute("""
  SELECT p.climate, COUNT(*) AS n_chars
  FROM characters c
  JOIN planets p ON c.homeworld = p.name
  GROUP BY p.climate
  ORDER BY n_chars DESC
""").fetchall()
)

[('Darth Vader', 'Sith', 'Tatooine', 'desert', 'Outer Rim'), ('Emperor Palpatine', 'Sith', 'Naboo', 'temperate', 'Mid Rim'), ('Han Solo', 'Rebel', 'Corellia', 'temperate', 'Core'), ('Leia Organa', 'Rebel', 'Alderaan', 'temperate', 'Core'), ('Luke Skywalker', 'Jedi', 'Tatooine', 'desert', 'Outer Rim'), ('Obi-Wan Kenobi', 'Jedi', 'Stewjon', 'temperate', 'Unknown'), ('Yoda', 'Jedi', 'Dagobah', 'swamp', 'Outer Rim')]
[('temperate', 4), ('desert', 2), ('swamp', 1)]


In [7]:
# Write a view of Jedi to Parquet
con.execute("""
  COPY (
    SELECT name, homeworld, midichlorians
    FROM characters
    WHERE affiliation = 'Jedi'
  ) TO 'jedi.parquet' (FORMAT 'parquet');
""")

# Query the Parquet file directly (no table needed)
print(
    con.execute("SELECT * FROM 'jedi.parquet' ORDER BY midichlorians DESC").fetchall()
)

[('Yoda', 'Dagobah', 17500), ('Obi-Wan Kenobi', 'Stewjon', 16000), ('Luke Skywalker', 'Tatooine', 15000)]
